In [33]:
from common import *

Structure : Done.


In [36]:
try:
    from bioservices import BioModels
except ImportError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "bioservices"])
    from bioservices import BioModels

s = BioModels()

WARNING [bioservices.BioModels:130]:  The URL (https://www.ebi.ac.uk/biomodels) provided cannot be reached.
INFO    [bioservices.BioModels:363]:  Initialising BioModels service (REST)


Creating directory /Users/guillaume.souede/Library/Caches/bioservices 


In [118]:
def create_query():
    query_parts_full = {
        'mode': '*:*',
        'species': 'TAXONOMY:9606',
        'curation_status': 'curationstatus:"Manually curated"',
        'formats': 'modelformat:"SBML"',
        'kw': 'submitter_keywords:"Immuno-oncology"'
    }

    query_parts = [value for value in query_parts_full.values() if value]
    query = " AND ".join(query_parts)

    query_parts_simple = {
        key: value.split(":")[1] if ":" in value else value
        for key, value in query_parts_full.items()
    }

    query_simple = " AND ".join(query_parts_simple.values())

    return query, query_simple

In [119]:
quer, squer = create_query()
print(f"Query: {BLUE}{quer}{RESET}")
print(f"Subquery: {BLUE}{squer}{RESET}")

Query: *:* AND TAXONOMY:9606 AND curationstatus:"Manually curated" AND modelformat:"SBML" AND submitter_keywords:"Immuno-oncology"
Subquery: * AND 9606 AND "Manually curated" AND "SBML" AND "Immuno-oncology"


In [120]:
def get_ids() -> list:
    query, sq = create_query()

    print(f"Generated query: {query}")

    try:
        search_results = s.search(query)

        if not search_results or 'models' not in search_results or not search_results['models']:
            print("No models found.")
            return []

        model_ids = [model['id'] for model in search_results['models']]
        return model_ids

    except Exception as e:
        print(f"Error during search: {e}")
        return []

In [45]:
def download_biomodels(directory: str, model_ids: list):
    filenames = []
    num_per_download = 100

    if not os.path.exists(directory):
        os.makedirs(directory)

    num_downloads = (len(model_ids) + num_per_download - 1) // num_per_download

    start = 0
    for download_number in range(1, num_downloads + 1):
        end = min(start + num_per_download, len(model_ids))
        fname = os.path.join(directory, f"Biomodels_{start+1}-{end}.zip")
        filenames.append(fname)

        if os.path.isfile(fname):
            os.remove(fname)

        current_batch_ids = model_ids[start:end]

        try:
            s.search_download(current_batch_ids, output_filename=fname)
            print(f"Biomodels models from ID {start+1} to {end} saved to {fname}")
        except Exception as e:
            print(f"Error downloading batch {start+1}-{end}: {e}")
            continue

        start = end

    # Consolidation
    consolidated_zip = os.path.join(directory, "biomodels.zip")
    with z.ZipFile(consolidated_zip, 'w') as zf_out:
        for fname in filenames:
            try:
                with z.ZipFile(fname, 'r') as zf_in:
                    for file_name in zf_in.namelist():
                        zf_out.writestr(file_name, zf_in.read(file_name))
            except Exception as e:
                print(f"Error consolidating {fname}: {e}")

    for fname in filenames:
        if os.path.isfile(fname):
            os.remove(fname)

    print(f"All models consolidated into {consolidated_zip}")
    return consolidated_zip

Using the code to answer :

In [47]:
ACTIVATE_DOWNLOAD = True
query, sq = create_query()
model_ids = get_ids()
print(f"{BLUE}Model IDs: {model_ids}.{RESET}")
print(f"{BLUE}Found {len(model_ids)} models matching the query.{RESET}")
if ACTIVATE_DOWNLOAD and model_ids:
    download_biomodels(modelsDir, model_ids)

Generated query: *:* AND TAXONOMY:9606 AND curationstatus:"Manually curated" AND modelformat:"SBML" AND submitter_keywords:"Immuno-oncology"
Model IDs: ['BIOMD0000000919', 'BIOMD0000000921', 'BIOMD0000001013', 'BIOMD0000001014', 'BIOMD0000001022', 'BIOMD0000001023', 'BIOMD0000001024', 'BIOMD0000001025', 'BIOMD0000001031', 'BIOMD0000001033'].
Found 10 models matching the query.


NameError: name 'modelsDir' is not defined

In [59]:
directory = "/Users/guillaume.souede/PycharmProjects/immunogit/models/BioModels/SBML"

# Call the function to download and consolidate the models
consolidated_zip = download_biomodels_from_api(directory)

# Check if the zip file was created successfully
if consolidated_zip:
    print(f"Models downloaded and consolidated into: {consolidated_zip}")
else:
    print("No models were downloaded.")

Status Code: 401
Response Content: 
Error querying BioModels: 401 Client Error: Unauthorized for url: https://www.ebi.ac.uk/biomodels/api/model/search?q=%2A%3A%2A+AND+curationstatus%3A%22Manually+curated%22+AND+modelformat%3A%22SBML%22+AND+TAXONOMY%3A9606&rows=10
No models were downloaded.
